# Assignment 9 - Answer

1.
Load the MNIST data, and split it into a training set, a validation set, and a test set (e.g., use 50,000 instances for training, 10,000 for validation, and 10,000 for testing). Then train various classifiers, such as a Random Forest classifier, a Logistic Regression classifier, an SVM, and a MLPClassifier (I haven't taught you yet, but its a simple neural network with multi-layers). 
 
Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?
 
Last, attemp to use XGBoost.  Does it improve the accuracy?

In [1]:
# Your code here

In [2]:
from sklearn.datasets import fetch_openml
import numpy as np

#fetching
mnist = fetch_openml('mnist_784', version=1)

#make sure is int
mnist.target = mnist.target.astype(np.int)

In [3]:
from sklearn.model_selection import train_test_split

#Load the MNIST data and split it into a training set, a validation set, and 
#a test set (e.g., use 50,000 instances for training, 10,000 for 
#validation, and 10,000 for testing)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    mnist.data, mnist.target, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=10000, random_state=42)

In [4]:
#define the classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

In [ ]:
models = [rnd_clf, log_clf, svm_clf, mlp_clf]
for model in models:
    print("Training the", model.__class__.__name__)
    model.fit(X_train, y_train)

In [ ]:
[model.score(X_val, y_val) for model in models]

Seems like Logistic Regression is hurting performance.  Not sure whether we should include it on the ensemble but we will think it later....Let's first define a Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

named_estimators = [
    ("rnd_clf", rnd_clf),
    ("log_clf", log_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]

voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)
voting_clf.score(X_val, y_val)

In [ ]:
#let's print out the individual estimator
[estimator.score(X_val, y_val) for estimator in voting_clf.estimators_]

Clearly, we can see that Logistic Regression is hurting the ensemble performance.  We can easily remove using del or fit again.  

In [ ]:
del voting_clf.estimators_[1]  #second estimator

#print estimators to make sure its deleted
voting_clf.estimators_

In [ ]:
#check the score again
voting_clf.score(X_val, y_val)  #yay increase a little!

By default, voting classifier is hard voting.  Let's try soft voting.  We can either fit again, but we can also easily set params to soft (this works because soft/hard is simply a function of all the models we already fitted)

In [ ]:
voting_clf.voting = "soft"
voting_clf.score(X_val, y_val)

Hmm...seems like hard wins....

Finally, let's try on our testing set which we should never touch it until the final testing

In [ ]:
voting_clf.voting = "hard"
print("Voting classifier score:", voting_clf.score(X_test, y_test))
print("Each classifier scor: ")
[estimator.score(X_test, y_test) for estimator in voting_clf.estimators_]

Hmm...it seems that the voting classifier only slighly increase the accuracy by little, when compared to the best estimator

let's try XGBoost

In [ ]:
import xgboost

xgb_clf = xgboost.XGBClassifier() 

#not improved after 2 iterations
xgb_clf.fit(X_train, y_train,
                eval_set=[(X_val, y_val)], early_stopping_rounds=2)
print("Score: ", xgb_clf.score(X_test, y_test))


2.
#implement Bagging ensembles of Decision Trees from scratch

"""
Coding considerations:
1. Load any data you would like to apply

2. Make sure to subsample *with* replacement.  I.e., 
   we should allow the same training instance to be subsampled 
   for the same predictor.  For example, X[5] can appear multiple
   times in the nth predictor (it may not even appear at all!)

3. Once all predictors are trained, the bagging ensemble can make a prediction.
   By simply aggregating, commonly using mode for classification and
   average for regression

"""

In [ ]:
# Your code here

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.3, shuffle=True, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import random
import numpy as np
from scipy import stats
from sklearn.metrics import classification_report

n_estimators = 5
boostrap_ratio = 0.1
tree_params = {'max_depth': 2, 'criterion':'gini', 'min_samples_split': 5}
models = [DecisionTreeClassifier(**tree_params) for _ in range(n_estimators)]

#sample size for each predictor
sample_size = int(boostrap_ratio * len(X_train))

#2 for x and y
xsamples = []
ysamples = []

#subsamples for each model
for _ in range(n_estimators):
    xsample = []
    ysample = []
    ##sampling with replacement; i.e., sample can occur more than once
    #for the same predictor
    for _ in range(sample_size):
        idx = random.randrange(X_train.shape[0])
        xsample.append(X_train[idx])
        ysample.append(y_train[idx])
    xsamples.append(xsample)
    ysamples.append(ysample)

#convert to numpy for easier manipulation
#usually if we start with list.append and later convert to np
#is faster than using np.empty
xsamples = np.asarray(xsamples)
ysamples = np.asarray(ysamples)

#fitting each estimator
for i, model in enumerate(models):
    _X = xsamples[i, :]
    _y = ysamples[i, :]
    model.fit(_X, _y)
    
#make prediction and return the probabilities
predictions = []
for model in models:
    y_pred = model.predict(X_test)
    predictions.append(y_pred)
    
#we can use unpacking technique to remove the outerlist
#we need [0] because .mode return both list and the counts
[y_pred] = stats.mode(predictions)[0]

print(classification_report(y_test, y_pred))


3. #implement AdaBoost for binary classification from scratch

"""
Coding considerations:

1. Load any binary classification problem you would like to apply

2. Given binary classification, AdaBoost expects input as -1 and 1

3. Once all predictors have been trained, the final predictions are
simply the np.sign of a_j * y_pred.  This works because if more
classifiers say that it is positive, then the sum will turn out
to be positive, thus we can simply look at the np.sign to know
the class

"""

In [ ]:
# Your code here

In [ ]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=500)
y = np.where(y==0,-1,1)  #change our y to be -1 if it is 0, otherwise 1

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score

n_samples = X_train.shape[0]
n_estimators = 20
tree_params = {'max_depth': 1}
models = [DecisionTreeClassifier(**tree_params) for _ in range(n_estimators)]

#initially, we set our weight to 1/n
W = np.ones(shape=n_samples) / n_samples

#keep collection of a_j
a_js = np.zeros(shape=n_estimators)

for j, model in enumerate(models):
    
    #train weak learner
    model.fit(X_train, y_train, sample_weight=W)
    
    #compute the errors r_j
    y_pred = model.predict(X_train) 
    compare = y_pred != y_train  #this works since True * w = 1 * w
#     compare = np.array([int(x) for x in (y_pred != y_train)])
    
    r_j = np.sum(W * compare) / sum(W)
    
    #compute the predictor weight a_j
    #let eta to be 1 as adaboost default 
    #if predictor is doing well, a_j will be big
    eta = 1
    a_j = eta * np.log ((1 - r_j) / r_j)
    a_js[j] = a_j
    
    #update sample weight; divide sum of W to normalize
    W = (W * np.exp(a_j)) / sum (W)
    
        
#make weighted predictions
Hx = 0
for i, model in enumerate(models):
    y_pred = model.predict(X_test)
    Hx += a_js[i] * y_pred
    
y_pred = np.sign(Hx)

print(classification_report(y_test, y_pred))



4.

#implement Gradient Boosting from scratch
#To add to the challenge, we shall create our algorithm that
#works for both binary classification and regression

"""

Coding considerations:

1. Load one binary classification and one regression problem you would like to apply

2. To make our algorithm work for both regression and binary classification
   We can perform this via a params called 'loss'
   if loss is defined as "mse", then its a regression problem and
   the residual errors shall be simply calculated as y - f(x)
   if loss is defined as "logistic", then its a classification problem
   and the residual errors shall be calculated as y - sigmoid(x)
   the reason of using sigmoid is obvious, since we want to create
   a mapping of x to something between 0 and 1 so it can be compared
   to y, thus we use sigmoid

3. You may wonder what classifier or regressor to use for this
   problem.  Luckily, if we define the loss function with sigmoid,
   we can simply use regressor as our estimator, since we already
   define sigmoid function that map any continuous value to values 
   between 0 and 1.  The only thing you have to do is to 
   create a tolerance function mapping >0.5 to 1 otherwise 0

4. When finding the loss, it is important to calculate the loss
   based on the total errors made by all models you have trained
   up to nth iteration
   
"""

In [ ]:
# Your code here

In [ ]:
from scipy.special import expit
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor

def mse(y, f):
    return y - f

def logit(y, f):
    return y - expit(f)

def fit(X, y, models, loss="mse"):
    
    if loss == "mse":
        loss_func = mse
    else:
        loss_func = logit
    
    models_trained = []
    
    #using DummyRegressor is a good technique for starting model
    first_model = DummyRegressor(strategy='constant', constant=0)
    first_model.fit(X, y)
    models_trained.append(first_model)
    
    #fit the estimators
    for i, model in enumerate(models):
        #predict using all the weak learners we trained up to
        #this point
        y_pred = predict(X, models_trained)
        
        #errors will be the total errors maded by models_trained
        residual = loss_func(y, y_pred)
        
        #fit the next model with residual
        model.fit(X, residual)
        
        models_trained.append(model)
        
    return models_trained
        
def predict(X, models):
    learning_rate = 0.1  ##hard code for now
    f0 = models[0].predict(X)  #first use the dummy model
    boosting = sum(learning_rate * model.predict(X) for model in models[1:])
    return f0 + boosting

def change_to_0_1(sample):
    return int(sample > 0.5)

def predict_class(X, models):
    probas = expit(predict(X, models))
    return np.array([change_to_0_1(proba) for proba in probas])

In [ ]:
# Regression

from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, random_state=42)

n_estimators = 200
tree_params = {'max_depth': 1}
models = [DecisionTreeRegressor(**tree_params) for _ in range(n_estimators)]

#fit the models
models = fit(X_train, y_train, models)

#predict
y_pred = predict(X_test, models)

#print metrics
print("Custom MSE: ", mean_squared_error(y_test, y_pred))


#Compare to sklearn: ls is the same as our mse
sklearn_model = GradientBoostingRegressor(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1,
    loss='ls'
)

y_pred_sk = sklearn_model.fit(X_train, y_train).predict(X_test)

#print metrics
print("Sklearn MSE: ", mean_squared_error(y_test, y_pred_sk))


In [ ]:
# Classification

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=42)

n_estimators = 200
tree_params = {'max_depth': 1}
models = [DecisionTreeRegressor(**tree_params) for _ in range(n_estimators)]

#fit the models
models = fit(X_train, y_train, models, loss="logistic")

#predict
y_pred = predict_class(X_test, models)

# #print metrics
print("Custom accuracy: ", accuracy_score(y_test, y_pred))

#Compare to sklearn: ls is the same as our mse
sklearn_model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1
)

y_pred_sk = sklearn_model.fit(X_train, y_train).predict(X_test)

#print metrics
print("Sklearn accuracy: ", accuracy_score(y_test, y_pred_sk))

